In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

%load_ext autoreload
%autoreload

In [2]:
from collections import defaultdict

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import ndcg_score
import lightgbm as lgb
from tqdm import tqdm

from src.data import get_sequences, load_data, download_movielens1m, ValSASRecDataset, TrainSASRecDataset
from src.utils import fix_seed
from src.model import SASRec


%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [7]:
seed = 0
max_len = 200
batch_size = 128
embedding_size = 64

n_ext_users = 10

In [6]:
download_movielens1m("data/movielens_1m")
ratings = load_data("data/movielens_1m")

ratings = ratings[ratings["rating"] > 3.5]

# Подготовка данных
num_users = ratings['user_id'].nunique()
num_items = ratings['item_id'].nunique()

user2id = {val:i for i, val in enumerate(ratings['user_id'].unique())}
item2id = {val:i+1 for i, val in enumerate(ratings['item_id'].unique())}

ratings['user_id'] = ratings['user_id'].map(user2id)
ratings['item_id'] = ratings['item_id'].map(item2id)

(train_sequences, val_sequences), (train_times, val_times), (train_users, val_users) = get_sequences(ratings)

Dataset already exists.


In [8]:
train_dataset = TrainSASRecDataset(train_sequences, train_times, max_len)
val_dataset = ValSASRecDataset(val_sequences, val_times, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [11]:
from torchsummary import summary
model = SASRec(num_items, embedding_size=embedding_size, max_len=max_len)


summary(model, (28))

TypeError: Input type is not recognized. Please ensure input_data is valid.
For multiple inputs to the network, ensure input_data passed in is a sequence of tensors or a list of tuple sizes. If you are having trouble here, please submit a GitHub issue.